In [ ]:
import openai
from langchain_openai import ChatOpenAI
from langchain.globals import set_debug

llm = ChatOpenAI( model_name="custom",
openai_api_key='3c52000cbd5f8bd87edfca29d9f8a102', 
openai_api_base='http://localhost:5000/v1',
# model_kwargs={
#     "logit_bias":{
#         # "320":-100,
#         # "2474":-100,
#         "20122":-100
#     }    
# },
temperature=0,
streaming=True,
verbose=True )


set_debug(True)

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from langchain.globals import set_debug
llm = Ollama(model="custom")
set_debug(True)

In [ ]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


In [ ]:
from langchain.tools.render import render_text_description

tools = [multiply, exponentiate, add]
tool_map = {tool.name: tool for tool in tools}
rendered_tools = render_text_description(tools)
rendered_tools

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""
System:You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.
Arguments is an dictionary of the arguments to pass to the tool.
Return only the JSON. Do not include any other text in your response.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "Question:{input} \n Response:")]
)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

chain = prompt | llm | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

In [ ]:
from typing import Union
from langchain_core.runnables import Runnable,RunnableLambda,RunnablePassthrough
def call_tool(args: dict) -> Union[str, Runnable]:
    """Function for dynamically constructing the end of the chain based on the model-selected tool."""

    tool = tool_map[args["name"]]
    return RunnablePassthrough.assign(output=itemgetter("arguments") | tool)
call_tool_list = RunnableLambda(call_tool).map()

In [ ]:
from operator import itemgetter

chain = prompt | llm | JsonOutputParser() | call_tool
chain.invoke({"input": "what's thirteen + one million"})